## Dataset Breast Cancer

### Load Dataset

In [86]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()

X = breast_cancer.data
y = breast_cancer.target

### Splitting Dataset

In [87]:
from sklearn.model_selection import train_test_split

X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Training Dataset
#### a. DecisionTreeClassifier
##### Training

In [88]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 10)

model.fit(X_train_set, y_train_set)

DecisionTreeClassifier(max_depth=10)

##### Evaluation

In [89]:
from sklearn.metrics import classification_report

y_prediction = model.predict(X_test_set)

print(classification_report(y_test_set, y_prediction))

              precision    recall  f1-score   support

           0       0.95      0.90      0.93        42
           1       0.95      0.97      0.96        72

    accuracy                           0.95       114
   macro avg       0.95      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114



#### b. Id3Estimator
##### Training

In [90]:
import six

from id3 import Id3Estimator

model_id3 = Id3Estimator()
model_id3 = model_id3.fit(X_train_set, y_train_set)

##### Evaluation

In [91]:
from sklearn.metrics import classification_report

y_prediction = model_id3.predict(X_test_set)

print(classification_report(y_test_set, y_prediction))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91        42
           1       0.92      0.99      0.95        72

    accuracy                           0.94       114
   macro avg       0.95      0.92      0.93       114
weighted avg       0.94      0.94      0.94       114



#### c. K Means 
##### Training

In [92]:
from sklearn.cluster import KMeans

model_kmeans = KMeans(n_clusters=2, random_state=0)

model_kmeans.fit(X_train_set)

KMeans(n_clusters=2, random_state=0)

##### Evaluation

In [93]:
from sklearn.metrics import classification_report

y_prediction = model_kmeans.predict(X_test_set)

print(classification_report(y_test_set, y_prediction))

              precision    recall  f1-score   support

           0       0.23      0.52      0.32        42
           1       0.00      0.00      0.00        72

    accuracy                           0.19       114
   macro avg       0.12      0.26      0.16       114
weighted avg       0.09      0.19      0.12       114



#### d. LogisticRegression
##### Training

In [76]:
from sklearn.linear_model import LogisticRegression

model_logistic = LogisticRegression(max_iter=10000)

model_logistic.fit(X_train_set, y_train_set)

LogisticRegression(max_iter=10000)

##### Evaluation

In [77]:
from sklearn.metrics import classification_report

y_prediction = model_logistic.predict(X_test_set)

print(classification_report(y_test_set, y_prediction))

              precision    recall  f1-score   support

           0       0.97      0.88      0.93        42
           1       0.93      0.99      0.96        72

    accuracy                           0.95       114
   macro avg       0.95      0.93      0.94       114
weighted avg       0.95      0.95      0.95       114



#### e. Neural_network
##### Training

In [209]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

clf = MLPClassifier(random_state=1).fit(X_train_set, y_train_set)

##### Evaluation

In [210]:
from sklearn.metrics import classification_report

y_prediction = clf.predict(X_test_set)

print(classification_report(y_test_set, y_prediction))

              precision    recall  f1-score   support

           0       0.97      0.88      0.93        42
           1       0.93      0.99      0.96        72

    accuracy                           0.95       114
   macro avg       0.95      0.93      0.94       114
weighted avg       0.95      0.95      0.95       114



#### f. SVM
##### Training

In [42]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

model_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))

model_svm.fit(X_train_set, y_train_set)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

##### Evaluation

In [43]:
from sklearn.metrics import classification_report

y_prediction = model_svm.predict(X_test_set)

print(classification_report(y_test_set, y_prediction))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96        42
           1       0.97      0.99      0.98        72

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



## Dataset Play-tennis

### Load Dataset

In [45]:
import pandas as pd

dataset_play_tennis = pd.read_csv("PlayTennis.csv")
dataset_play_tennis_data = dataset_play_tennis[{'Outlook', 'Temperature', 'Humidity', 'Wind'}]
dataset_play_tennis_target = dataset_play_tennis[{'Play Tennis'}]

# dataset_play_tennis_target
# dataset_play_tennis_data

### Label Encoder

##### Humidity : 0 = "High", 1 = "Normal"
##### Temperature : 0 = "Cool", 1 = "Hot", 2 = "Mild"
##### Outlook : 0 = "Overcast", 1 = "Rain", 2 = "Sunny"
##### Wind : 0 = "Strong", 1 = "Weak"
##### Play Tennis : 0 = "No", 1 = "Yes"

In [46]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

x_data_encoded = dataset_play_tennis_data.copy()
if dataset_play_tennis_data is not None:
    for column in dataset_play_tennis_data.columns:
        x_data_encoded[column] = le.fit_transform(x_data_encoded[column])

y_target_encoded = dataset_play_tennis_target.copy()
if dataset_play_tennis_target is not None:
    for column in dataset_play_tennis_target.columns:
        y_target_encoded[column] = le.fit_transform(y_target_encoded[column]) 

### Splitting Data

In [47]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data_encoded, y_target_encoded, test_size=0.2, random_state=1)

### Training Dataset
#### a. DecisionTreeClassifier
##### Training

In [48]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 10)

model.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=10)

In [49]:
from sklearn.metrics import classification_report

y_prediction = model.predict(x_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3



#### b. Id3Estimator
##### Training

In [50]:
from id3 import Id3Estimator

model_id3_pandas = Id3Estimator()
model_id3_pandas = model_id3_pandas.fit(x_train, y_train.values.ravel())

##### Evaluation

In [51]:
from sklearn.metrics import classification_report

y_prediction = model_id3_pandas.predict(x_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3



#### c. K Means 
##### Training

In [82]:
from sklearn.cluster import KMeans

model_kmeans_pandas = KMeans(n_clusters=2, random_state=0)

model_kmeans_pandas.fit(x_train)

KMeans(n_clusters=2, random_state=0)

##### Evaluation

In [85]:
from sklearn.metrics import classification_report

y_prediction = model_kmeans_pandas.predict(x_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



#### d. LogisticRegression
##### Training

In [95]:
from sklearn.linear_model import LogisticRegression

model_logistic_pandas = LogisticRegression(max_iter=10000)

model_logistic_pandas.fit(x_train, y_train.values.ravel())

LogisticRegression(max_iter=10000)

##### Evaluation

In [102]:
from sklearn.metrics import classification_report

y_prediction = model_logistic_pandas.predict(x_test)

print(classification_report(y_test, y_prediction, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



#### e. Neural_network
##### Training

In [103]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

clf_pandas = MLPClassifier(random_state=1, max_iter=700).fit(x_train, y_train.values.ravel())

##### Evaluation

In [106]:
from sklearn.metrics import classification_report

y_prediction = clf_pandas.predict(x_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



#### f. SVM
##### Training

In [65]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

model_svm_pandas = make_pipeline(StandardScaler(), SVC(gamma='auto'))

model_svm_pandas.fit(x_train, y_train.values.ravel())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

##### Evaluation

In [66]:
from sklearn.metrics import classification_report

y_prediction = model_svm_pandas.predict(x_test)

print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

